In [1]:
from __future__ import print_function, division
#Allows relative imports
import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
#imports from files
from src.preprocessing import *
from src.train_eval_helpers import *
from src.VAE_train import *
from src.vautoencoders import *
#from loss_metrics import *
from src.plots import *
from src.pickling import *
from src.datasets import *

import pandas as pd 
import numpy as np
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

#checking gpu status
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("Using : {}".format(device))
else:
    device = torch.device('cpu')
    print("Using : {}".format(device))
    
#Plot and stuff
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi']= 200
sns.set_style('darkgrid')


# Ignore warnings)
import warnings
warnings.filterwarnings("ignore")
    
%load_ext autoreload
%autoreload 2

Using : cuda


In [2]:
batch2 = pd.read_csv('../training_data_new/emerson_raw/batch2/emerson_batch2_filtered_length_vdj.tsv',
                     sep='\t')
deepcat = pd.read_csv('./normal.txt', header=None)

In [7]:
batch2.query('amino_acid in @deepcat.amino_acid.values')

,amino_acid,frequency,v_family,TRBV,d_family,d_gene,j_family,TRBJ,filename,age,sex,cohort,race,pred_cmv,true_cmv,len
12,CASSVPGQPQHF,0.023721,TCRBV13,TCRBV13-01,TCRBD01,TCRBD01-01,TCRBJ01,TCRBJ01-05,Keck0001_MC1.tsv,22,Female,2,Caucasian,0,1,12
17,CASSLSRGGVYEQYF,0.000715,TCRBV27,TCRBV27-01,TCRBD02,TCRBD02-01,TCRBJ02,TCRBJ02-07,Keck0001_MC1.tsv,22,Female,2,Caucasian,0,1,15
44,CASSLTGGEQYF,0.000006,TCRBV28,TCRBV28-01,TCRBD01,TCRBD01-01,TCRBJ02,TCRBJ02-07,Keck0001_MC1.tsv,22,Female,2,Caucasian,0,1,12
46,CASSGGLSYEQYF,0.000006,TCRBV27,TCRBV27-01,TCRBD02,TCRBD02-01,TCRBJ02,TCRBJ02-07,Keck0001_MC1.tsv,22,Female,2,Caucasian,0,1,13
53,CASSYGTGNYEQYF,0.000006,TCRBV27,TCRBV27-01,TCRBD01,TCRBD01-01,TCRBJ02,TCRBJ02-07,Keck0001_MC1.tsv,22,Female,2,Caucasian,0,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12569056,CASRPTGLNTEAFF,0.000002,TCRBV19,TCRBV19-01,TCRBD01,TCRBD01-01,TCRBJ01,TCRBJ01-01,Keck0120_MC1.tsv,26,Female,2,Caucasian,0,0,14
12569066,CASSPPSVNTEAFF,0.000002,TCRBV18,TCRBV18-01,TCRBD01,TCRBD01-01,TCRBJ01,TCRBJ01-01,Keck0120_MC1.tsv,26,Female,2,Caucasian,0,0,14
12569067,CASSLAVNTEAFF,0.000002,TCRBV07,TCRBV07-08,TCRBD01,TCRBD01-01,TCRBJ01,TCRBJ01-01,Keck0120_MC1.tsv,26,Female,2,Caucasian,0,0,13
12569094,CASSESGGNTEAFF,0.000002,TCRBV10,TCRBV10-01,TCRBD02,TCRBD02-01,TCRBJ01,TCRBJ01-01,Keck0120_MC1.tsv,26,Female,2,Caucasian,0,0,14
